<a href="https://colab.research.google.com/github/drscicomm/food-security/blob/main/FA_tweets_scrape_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

From @ImpossibleFoods Tweets: Scraping 2020

Code adapted from: https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a

## Set up libraries

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [ ]:
# Twitter API Authentication with OAuth2 Bearer Token
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAO6QNAEAAAAAJ7Sqp1gnSH2GE0f69v9LkgwtYNg%3D3HIrC1yFpVJtQDQbi6YnVUiGkEJVGUwRV9EgY4lae7Y2GqolRU'


In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [ ]:
def create_url(keyword, start_date, end_date, max_results = 500):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics',
                    'next_token': {}}
    return (search_url, query_params)

In [ ]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

#Make CSV file and define append to csv function

In [ ]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
       # if ('geo' in tweet):   
           # geo = tweet['geo']['place_id']
      #  else:
           # geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [author_id, created_at, tweet_id, lang, like_count, quote_count, reply_count, retweet_count, source, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

# Looping

In [ ]:
#Inputs for tweets Jan 2020
headers = create_headers(bearer_token)
keyword = "-is:retweet -RT food emergency OR food crisis OR food aid OR food need OR food corona OR food rona OR food assistance OR food distribution OR food distribute OR food free OR WIC OR food security OR food insecurity OR food hungry OR #freefood OR #mealservice OR food pantry OR food bank OR covid food"
start_list =    ['2020-01-01T00:00:00.000Z',
                 '2020-01-02T00:00:00.000Z',
                 '2020-01-03T00:00:00.000Z',
                 '2020-01-04T00:00:00.000Z',
                 '2020-01-05T00:00:00.000Z',
                 '2020-01-06T00:00:00.000Z',
                 '2020-01-07T00:00:00.000Z',
                 '2020-01-08T00:00:00.000Z',
                 '2020-01-09T00:00:00.000Z',
                 '2020-01-10T00:00:00.000Z',
                 '2020-01-11T00:00:00.000Z',
                 '2020-01-12T00:00:00.000Z',
                 '2020-01-13T00:00:00.000Z',
                 '2020-01-14T00:00:00.000Z',
                 '2020-01-15T00:00:00.000Z',
                 '2020-01-16T00:00:00.000Z',
                 '2020-01-17T00:00:00.000Z',
                 '2020-01-18T00:00:00.000Z',
                 '2020-01-19T00:00:00.000Z',
                 '2020-01-20T00:00:00.000Z',
                 '2020-01-21T00:00:00.000Z',
                 '2020-01-22T00:00:00.000Z',
                 '2020-01-23T00:00:00.000Z',
                 '2020-01-24T00:00:00.000Z',
                 '2020-01-25T00:00:00.000Z',
                 '2020-01-26T00:00:00.000Z',
                 '2020-01-27T00:00:00.000Z',
                 '2020-01-28T00:00:00.000Z',
                 '2020-01-29T00:00:00.000Z',
                 '2020-01-30T00:00:00.000Z',
                 '2020-01-31T00:00:00.000Z']

end_list =      ['2020-01-01T23:59:59.000Z',
                 '2020-01-02T23:59:59.000Z',
                 '2020-01-03T23:59:59.000Z',
                 '2020-01-04T23:59:59.000Z',
                 '2020-01-05T23:59:59.000Z',
                 '2020-01-06T23:59:59.000Z',
                 '2020-01-07T23:59:59.000Z',
                 '2020-01-08T23:59:59.000Z',
                 '2020-01-09T23:59:59.000Z',
                 '2020-01-10T23:59:59.000Z',
                 '2020-01-11T23:59:59.000Z',
                 '2020-01-12T23:59:59.000Z',
                 '2020-01-13T23:59:59.000Z',
                 '2020-01-14T23:59:59.000Z',
                 '2020-01-15T23:59:59.000Z',
                 '2020-01-16T23:59:59.000Z',
                 '2020-01-17T23:59:59.000Z',
                 '2020-01-18T23:59:59.000Z',
                 '2020-01-19T23:59:59.000Z',
                 '2020-01-20T23:59:59.000Z',
                 '2020-01-21T23:59:59.000Z',
                 '2020-01-22T23:59:59.000Z',
                 '2020-01-23T23:59:59.000Z',
                 '2020-01-24T23:59:59.000Z',
                 '2020-01-25T23:59:59.000Z',
                 '2020-01-26T23:59:59.000Z',
                 '2020-01-27T23:59:59.000Z',
                 '2020-01-28T23:59:59.000Z',
                 '2020-01-29T23:59:59.000Z',
                 '2020-01-30T23:59:59.000Z',
                 '2020-01-31T23:59:59.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("fromtweets-jan2020.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-jan2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-jan2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)


In [ ]:
#Inputs for tweets Feb 2020
headers = create_headers(bearer_token)
keyword = "-is:retweet -RT food emergency OR food crisis OR food aid OR food need OR food corona OR food rona OR food assistance OR food distribution OR food distribute OR food free OR WIC OR food security OR food insecurity OR food hungry OR #freefood OR #mealservice OR food pantry OR food bank OR covid food"
start_list =    ['2020-02-01T00:00:00.000Z',
                 '2020-02-02T00:00:00.000Z',
                 '2020-02-03T00:00:00.000Z',
                 '2020-02-04T00:00:00.000Z',
                 '2020-02-05T00:00:00.000Z',
                 '2020-02-06T00:00:00.000Z',
                 '2020-02-07T00:00:00.000Z',
                 '2020-02-08T00:00:00.000Z',
                 '2020-02-09T00:00:00.000Z',
                 '2020-02-10T00:00:00.000Z',
                 '2020-02-11T00:00:00.000Z',
                 '2020-02-12T00:00:00.000Z',
                 '2020-02-13T00:00:00.000Z',
                 '2020-02-14T00:00:00.000Z',
                 '2020-02-15T00:00:00.000Z',
                 '2020-02-16T00:00:00.000Z',
                 '2020-02-17T00:00:00.000Z',
                 '2020-02-18T00:00:00.000Z',
                 '2020-02-19T00:00:00.000Z',
                 '2020-02-20T00:00:00.000Z',
                 '2020-02-21T00:00:00.000Z',
                 '2020-02-22T00:00:00.000Z',
                 '2020-02-23T00:00:00.000Z',
                 '2020-02-24T00:00:00.000Z',
                 '2020-02-25T00:00:00.000Z',
                 '2020-02-26T00:00:00.000Z',
                 '2020-02-27T00:00:00.000Z',
                 '2020-02-28T00:00:00.000Z']

end_list =      ['2020-02-01T23:59:59.000Z',
                 '2020-02-02T23:59:59.000Z',
                 '2020-02-03T23:59:59.000Z',
                 '2020-02-04T23:59:59.000Z',
                 '2020-02-05T23:59:59.000Z',
                 '2020-02-06T23:59:59.000Z',
                 '2020-02-07T23:59:59.000Z',
                 '2020-02-08T23:59:59.000Z',
                 '2020-02-09T23:59:59.000Z',
                 '2020-02-10T23:59:59.000Z',
                 '2020-02-11T23:59:59.000Z',
                 '2020-02-12T23:59:59.000Z',
                 '2020-02-13T23:59:59.000Z',
                 '2020-02-14T23:59:59.000Z',
                 '2020-02-15T23:59:59.000Z',
                 '2020-02-16T23:59:59.000Z',
                 '2020-02-17T23:59:59.000Z',
                 '2020-02-18T23:59:59.000Z',
                 '2020-02-19T23:59:59.000Z',
                 '2020-02-20T23:59:59.000Z',
                 '2020-02-21T23:59:59.000Z',
                 '2020-02-22T23:59:59.000Z',
                 '2020-02-23T23:59:59.000Z',
                 '2020-02-24T23:59:59.000Z',
                 '2020-02-25T23:59:59.000Z',
                 '2020-02-26T23:59:59.000Z',
                 '2020-02-27T23:59:59.000Z',
                 '2020-02-28T23:59:59.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("fromtweets-feb2020.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-feb2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-feb2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71f2wkg83fahuhwmxtuqrcq285ml
Start Date:  2020-02-01T00:00:00.000Z
# of Tweets added from this response:  483
Total # of Tweets added:  483
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71f2wkg83fahuhwmxtuqrcq285ml
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71f2wkfvy8fm23o0kqdjb592a0zh
Start Date:  2020-02-01T00:00:00.000Z
# of Tweets added from this response:  490
Total # of Tweets added:  973
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71fhpd4jpwvqrfyup8njy2ucrtz1
Start Date:  2020-02-02T00:00:00.000Z
# of Tweets added from this response:  491
Total # of Tweets added:  1464
-------------------
-------------------
Token:  b26v89c19zqg8o3fo71fhpd4jpwvqrfyup8njy2ucrtz1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo71fhpctstdhawur42vcfh9ymf5a5
Start Date:  2020-02-02T00:00:00.000Z
# of T

In [ ]:
#Inputs for tweets March 2020
headers = create_headers(bearer_token)
keyword = "-is:retweet -RT food emergency OR food crisis OR food aid OR food need OR food corona OR food rona OR food assistance OR food distribution OR food distribute OR food free OR WIC OR food security OR food insecurity OR food hungry OR #freefood OR #mealservice OR food pantry OR food bank OR covid food"
start_list =    ['2020-03-01T00:00:00.000Z',
                 '2020-03-02T00:00:00.000Z',
                 '2020-03-03T00:00:00.000Z',
                 '2020-03-04T00:00:00.000Z',
                 '2020-03-05T00:00:00.000Z',
                 '2020-03-06T00:00:00.000Z',
                 '2020-03-07T00:00:00.000Z',
                 '2020-03-08T00:00:00.000Z',
                 '2020-03-09T00:00:00.000Z',
                 '2020-03-10T00:00:00.000Z',
                 '2020-03-11T00:00:00.000Z',
                 '2020-03-12T00:00:00.000Z',
                 '2020-03-13T00:00:00.000Z',
                 '2020-03-14T00:00:00.000Z',
                 '2020-03-15T00:00:00.000Z',
                 '2020-03-16T00:00:00.000Z',
                 '2020-03-17T00:00:00.000Z',
                 '2020-03-18T00:00:00.000Z',
                 '2020-03-19T00:00:00.000Z',
                 '2020-03-20T00:00:00.000Z',
                 '2020-03-21T00:00:00.000Z',
                 '2020-03-22T00:00:00.000Z',
                 '2020-03-23T00:00:00.000Z',
                 '2020-03-24T00:00:00.000Z',
                 '2020-03-25T00:00:00.000Z',
                 '2020-03-26T00:00:00.000Z',
                 '2020-03-27T00:00:00.000Z',
                 '2020-03-28T00:00:00.000Z',
                 '2020-03-29T00:00:00.000Z',
                 '2020-03-30T00:00:00.000Z',
                 '2020-03-31T00:00:00.000Z']

end_list =      ['2020-03-01T23:59:59.000Z',
                 '2020-03-02T23:59:59.000Z',
                 '2020-03-03T23:59:59.000Z',
                 '2020-03-04T23:59:59.000Z',
                 '2020-03-05T23:59:59.000Z',
                 '2020-03-06T23:59:59.000Z',
                 '2020-03-07T23:59:59.000Z',
                 '2020-03-08T23:59:59.000Z',
                 '2020-03-09T23:59:59.000Z',
                 '2020-03-10T23:59:59.000Z',
                 '2020-03-11T23:59:59.000Z',
                 '2020-03-12T23:59:59.000Z',
                 '2020-03-13T23:59:59.000Z',
                 '2020-03-14T23:59:59.000Z',
                 '2020-03-15T23:59:59.000Z',
                 '2020-03-16T23:59:59.000Z',
                 '2020-03-17T23:59:59.000Z',
                 '2020-03-18T23:59:59.000Z',
                 '2020-03-19T23:59:59.000Z',
                 '2020-03-20T23:59:59.000Z',
                 '2020-03-21T23:59:59.000Z',
                 '2020-03-22T23:59:59.000Z',
                 '2020-03-23T23:59:59.000Z',
                 '2020-03-24T23:59:59.000Z',
                 '2020-03-25T23:59:59.000Z',
                 '2020-03-26T23:59:59.000Z',
                 '2020-03-27T23:59:59.000Z',
                 '2020-03-28T23:59:59.000Z',
                 '2020-03-29T23:59:59.000Z',
                 '2020-03-30T23:59:59.000Z',
                 '2020-03-31T23:59:59.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("fromtweets-march2020.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-march2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-march2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo74f9nkjpfnb81bwm5hqo3jwkfq7x
Start Date:  2020-03-01T00:00:00.000Z
# of Tweets added from this response:  491
Total # of Tweets added:  491
-------------------
-------------------
Token:  b26v89c19zqg8o3fo74f9nkjpfnb81bwm5hqo3jwkfq7x
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo74f9nk94kbdqmj2g9fpup5x54qd9
Start Date:  2020-03-01T00:00:00.000Z
# of Tweets added from this response:  482
Total # of Tweets added:  973
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo74f9w29830xwxo2nb07yc0jjief1
Start Date:  2020-03-02T00:00:00.000Z
# of Tweets added from this response:  476
Total # of Tweets added:  1449
-------------------
-------------------
Token:  b26v89c19zqg8o3fo74f9w29830xwxo2nb07yc0jjief1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo74f9w1yn7hhajgmtx89e1z5q1oql
Start Date:  2020-03-02T00:00:00.000Z
# of T

In [ ]:
#Inputs for tweets April 2020
headers = create_headers(bearer_token)
keyword = "-is:retweet -RT food emergency OR food crisis OR food aid OR food need OR food corona OR food rona OR food assistance OR food distribution OR food distribute OR food free OR WIC OR food security OR food insecurity OR food hungry OR #freefood OR #mealservice OR food pantry OR food bank OR covid food"
start_list =    ['2020-04-01T00:00:00.000Z',
                 '2020-04-02T00:00:00.000Z',
                 '2020-04-03T00:00:00.000Z',
                 '2020-04-04T00:00:00.000Z',
                 '2020-04-05T00:00:00.000Z',
                 '2020-04-06T00:00:00.000Z',
                 '2020-04-07T00:00:00.000Z',
                 '2020-04-08T00:00:00.000Z',
                 '2020-04-09T00:00:00.000Z',
                 '2020-04-10T00:00:00.000Z',
                 '2020-04-11T00:00:00.000Z',
                 '2020-04-12T00:00:00.000Z',
                 '2020-04-13T00:00:00.000Z',
                 '2020-04-14T00:00:00.000Z',
                 '2020-04-15T00:00:00.000Z',
                 '2020-04-16T00:00:00.000Z',
                 '2020-04-17T00:00:00.000Z',
                 '2020-04-18T00:00:00.000Z',
                 '2020-04-19T00:00:00.000Z',
                 '2020-04-20T00:00:00.000Z',
                 '2020-04-21T00:00:00.000Z',
                 '2020-04-22T00:00:00.000Z',
                 '2020-04-23T00:00:00.000Z',
                 '2020-04-24T00:00:00.000Z',
                 '2020-04-25T00:00:00.000Z',
                 '2020-04-26T00:00:00.000Z',
                 '2020-04-27T00:00:00.000Z',
                 '2020-04-28T00:00:00.000Z',
                 '2020-04-29T00:00:00.000Z',
                 '2020-04-30T00:00:00.000Z']

end_list =      ['2020-04-01T23:59:59.000Z',
                 '2020-04-02T23:59:59.000Z',
                 '2020-04-03T23:59:59.000Z',
                 '2020-04-04T23:59:59.000Z',
                 '2020-04-05T23:59:59.000Z',
                 '2020-04-06T23:59:59.000Z',
                 '2020-04-07T23:59:59.000Z',
                 '2020-04-08T23:59:59.000Z',
                 '2020-04-09T23:59:59.000Z',
                 '2020-04-10T23:59:59.000Z',
                 '2020-04-11T23:59:59.000Z',
                 '2020-04-12T23:59:59.000Z',
                 '2020-04-13T23:59:59.000Z',
                 '2020-04-14T23:59:59.000Z',
                 '2020-04-15T23:59:59.000Z',
                 '2020-04-16T23:59:59.000Z',
                 '2020-04-17T23:59:59.000Z',
                 '2020-04-18T23:59:59.000Z',
                 '2020-04-19T23:59:59.000Z',
                 '2020-04-20T23:59:59.000Z',
                 '2020-04-21T23:59:59.000Z',
                 '2020-04-22T23:59:59.000Z',
                 '2020-04-23T23:59:59.000Z',
                 '2020-04-24T23:59:59.000Z',
                 '2020-04-25T23:59:59.000Z',
                 '2020-04-26T23:59:59.000Z',
                 '2020-04-27T23:59:59.000Z',
                 '2020-04-28T23:59:59.000Z',
                 '2020-04-29T23:59:59.000Z',
                 '2020-04-30T23:59:59.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("fromtweets-april2020.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-april2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-april2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo77fgti1orfxe0opmfd2gk3ceizct
Start Date:  2020-04-01T00:00:00.000Z
# of Tweets added from this response:  489
Total # of Tweets added:  489
-------------------
-------------------
Token:  b26v89c19zqg8o3fo77fgti1orfxe0opmfd2gk3ceizct
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo77fgti1lqes8uk9pbvf3lu822ef1
Start Date:  2020-04-01T00:00:00.000Z
# of Tweets added from this response:  488
Total # of Tweets added:  977
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo77fh201m7fy4fssk2kl79vdj9htp
Start Date:  2020-04-02T00:00:00.000Z
# of Tweets added from this response:  489
Total # of Tweets added:  1466
-------------------
-------------------
Token:  b26v89c19zqg8o3fo77fh201m7fy4fssk2kl79vdj9htp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo77fh201kp1d6utksx69m8km5jaf1
Start Date:  2020-04-02T00:00:00.000Z
# of T

In [ ]:
#Inputs for tweets May 2020
headers = create_headers(bearer_token)
keyword = "-is:retweet -RT food emergency OR food crisis OR food aid OR food need OR food corona OR food rona OR food assistance OR food distribution OR food distribute OR food free OR WIC OR food security OR food insecurity OR food hungry OR #freefood OR #mealservice OR food pantry OR food bank OR covid food"
start_list =    ['2020-05-01T00:00:00.000Z',
                 '2020-05-02T00:00:00.000Z',
                 '2020-05-03T00:00:00.000Z',
                 '2020-05-04T00:00:00.000Z',
                 '2020-05-05T00:00:00.000Z',
                 '2020-05-06T00:00:00.000Z',
                 '2020-05-07T00:00:00.000Z',
                 '2020-05-08T00:00:00.000Z',
                 '2020-05-09T00:00:00.000Z',
                 '2020-05-10T00:00:00.000Z',
                 '2020-05-11T00:00:00.000Z',
                 '2020-05-12T00:00:00.000Z',
                 '2020-05-13T00:00:00.000Z',
                 '2020-05-14T00:00:00.000Z',
                 '2020-05-15T00:00:00.000Z',
                 '2020-05-16T00:00:00.000Z',
                 '2020-05-17T00:00:00.000Z',
                 '2020-05-18T00:00:00.000Z',
                 '2020-05-19T00:00:00.000Z',
                 '2020-05-20T00:00:00.000Z',
                 '2020-05-21T00:00:00.000Z',
                 '2020-05-22T00:00:00.000Z',
                 '2020-05-23T00:00:00.000Z',
                 '2020-05-24T00:00:00.000Z',
                 '2020-05-25T00:00:00.000Z',
                 '2020-05-26T00:00:00.000Z',
                 '2020-05-27T00:00:00.000Z',
                 '2020-05-28T00:00:00.000Z',
                 '2020-05-29T00:00:00.000Z',
                 '2020-05-30T00:00:00.000Z',
                 '2020-05-31T00:00:00.000Z']

end_list =      ['2020-05-01T23:59:59.000Z',
                 '2020-05-02T23:59:59.000Z',
                 '2020-05-03T23:59:59.000Z',
                 '2020-05-04T23:59:59.000Z',
                 '2020-05-05T23:59:59.000Z',
                 '2020-05-06T23:59:59.000Z',
                 '2020-05-07T23:59:59.000Z',
                 '2020-05-08T23:59:59.000Z',
                 '2020-05-09T23:59:59.000Z',
                 '2020-05-10T23:59:59.000Z',
                 '2020-05-11T23:59:59.000Z',
                 '2020-05-12T23:59:59.000Z',
                 '2020-05-13T23:59:59.000Z',
                 '2020-05-14T23:59:59.000Z',
                 '2020-05-15T23:59:59.000Z',
                 '2020-05-16T23:59:59.000Z',
                 '2020-05-17T23:59:59.000Z',
                 '2020-05-18T23:59:59.000Z',
                 '2020-05-19T23:59:59.000Z',
                 '2020-05-20T23:59:59.000Z',
                 '2020-05-21T23:59:59.000Z',
                 '2020-05-22T23:59:59.000Z',
                 '2020-05-23T23:59:59.000Z',
                 '2020-05-24T23:59:59.000Z',
                 '2020-05-25T23:59:59.000Z',
                 '2020-05-26T23:59:59.000Z',
                 '2020-05-27T23:59:59.000Z',
                 '2020-05-28T23:59:59.000Z',
                 '2020-05-29T23:59:59.000Z',
                 '2020-05-30T23:59:59.000Z',
                 '2020-05-31T23:59:59.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("fromtweets-may2020.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-may2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-may2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7afnszk5ggs4l4q40m6t7hjnukcd
Start Date:  2020-05-01T00:00:00.000Z
# of Tweets added from this response:  477
Total # of Tweets added:  477
-------------------
-------------------
Token:  b26v89c19zqg8o3fo7afnszk5ggs4l4q40m6t7hjnukcd
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7afnszk0wt863zwwxqvkduvyw2gt
Start Date:  2020-05-01T00:00:00.000Z
# of Tweets added from this response:  463
Total # of Tweets added:  940
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7afo1hk1cklonebuvpcdtbwbup31
Start Date:  2020-05-02T00:00:00.000Z
# of Tweets added from this response:  485
Total # of Tweets added:  1425
-------------------
-------------------
Token:  b26v89c19zqg8o3fo7afo1hk1cklonebuvpcdtbwbup31
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7afo1h9jj4kiu8m48zacsuhfz4al
Start Date:  2020-05-02T00:00:00.000Z
# of T

In [ ]:
#Inputs for tweets June 2020
headers = create_headers(bearer_token)
keyword = "-is:retweet -RT food emergency OR food crisis OR food aid OR food need OR food corona OR food rona OR food assistance OR food distribution OR food distribute OR food free OR WIC OR food security OR food insecurity OR food hungry OR #freefood OR #mealservice OR food pantry OR food bank OR covid food"
start_list =    ['2020-06-01T00:00:00.000Z',
                 '2020-06-02T00:00:00.000Z',
                 '2020-06-03T00:00:00.000Z',
                 '2020-06-04T00:00:00.000Z',
                 '2020-06-05T00:00:00.000Z',
                 '2020-06-06T00:00:00.000Z',
                 '2020-06-07T00:00:00.000Z',
                 '2020-06-08T00:00:00.000Z',
                 '2020-06-09T00:00:00.000Z',
                 '2020-06-10T00:00:00.000Z',
                 '2020-06-11T00:00:00.000Z',
                 '2020-06-12T00:00:00.000Z',
                 '2020-06-13T00:00:00.000Z',
                 '2020-06-14T00:00:00.000Z',
                 '2020-06-15T00:00:00.000Z',
                 '2020-06-16T00:00:00.000Z',
                 '2020-06-17T00:00:00.000Z',
                 '2020-06-18T00:00:00.000Z',
                 '2020-06-19T00:00:00.000Z',
                 '2020-06-20T00:00:00.000Z',
                 '2020-06-21T00:00:00.000Z',
                 '2020-06-22T00:00:00.000Z',
                 '2020-06-23T00:00:00.000Z',
                 '2020-06-24T00:00:00.000Z',
                 '2020-06-25T00:00:00.000Z',
                 '2020-06-26T00:00:00.000Z',
                 '2020-06-27T00:00:00.000Z',
                 '2020-06-28T00:00:00.000Z',
                 '2020-06-29T00:00:00.000Z',
                 '2020-06-30T00:00:00.000Z']

end_list =      ['2020-06-01T23:59:59.000Z',
                 '2020-06-02T23:59:59.000Z',
                 '2020-06-03T23:59:59.000Z',
                 '2020-06-04T23:59:59.000Z',
                 '2020-06-05T23:59:59.000Z',
                 '2020-06-06T23:59:59.000Z',
                 '2020-06-07T23:59:59.000Z',
                 '2020-06-08T23:59:59.000Z',
                 '2020-06-09T23:59:59.000Z',
                 '2020-06-10T23:59:59.000Z',
                 '2020-06-11T23:59:59.000Z',
                 '2020-06-12T23:59:59.000Z',
                 '2020-06-13T23:59:59.000Z',
                 '2020-06-14T23:59:59.000Z',
                 '2020-06-15T23:59:59.000Z',
                 '2020-06-16T23:59:59.000Z',
                 '2020-06-17T23:59:59.000Z',
                 '2020-06-18T23:59:59.000Z',
                 '2020-06-19T23:59:59.000Z',
                 '2020-06-20T23:59:59.000Z',
                 '2020-06-21T23:59:59.000Z',
                 '2020-06-22T23:59:59.000Z',
                 '2020-06-23T23:59:59.000Z',
                 '2020-06-24T23:59:59.000Z',
                 '2020-06-25T23:59:59.000Z',
                 '2020-06-26T23:59:59.000Z',
                 '2020-06-27T23:59:59.000Z',
                 '2020-06-28T23:59:59.000Z',
                 '2020-06-29T23:59:59.000Z',
                 '2020-06-30T23:59:59.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("fromtweets-june2020.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-june2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-june2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7dfv0z2o25xzgmwnjx0r5nfjp7y5
Start Date:  2020-06-01T00:00:00.000Z
# of Tweets added from this response:  480
Total # of Tweets added:  480
-------------------
-------------------
Token:  b26v89c19zqg8o3fo7dfv0z2o25xzgmwnjx0r5nfjp7y5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7dfv0z2l1zk0414cd0csvait1gu5
Start Date:  2020-06-01T00:00:00.000Z
# of Tweets added from this response:  477
Total # of Tweets added:  957
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7dfv7f1xp196v6mjwhuvqqxbl30d
Start Date:  2020-06-02T00:00:00.000Z
# of Tweets added from this response:  489
Total # of Tweets added:  1446
-------------------
-------------------
Token:  b26v89c19zqg8o3fo7dfv7f1xp196v6mjwhuvqqxbl30d
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7dfv7f1w5kaq9uqu6ntpw51r5z7h
Start Date:  2020-06-02T00:00:00.000Z
# of T

In [ ]:
#Inputs for tweets July 2020
headers = create_headers(bearer_token)
keyword = "-is:retweet -RT food emergency OR food crisis OR food aid OR food need OR food corona OR food rona OR food assistance OR food distribution OR food distribute OR food free OR WIC OR food security OR food insecurity OR food hungry OR #freefood OR #mealservice OR food pantry OR food bank OR covid food"
start_list =    ['2020-07-01T00:00:00.000Z',
                 '2020-07-02T00:00:00.000Z',
                 '2020-07-03T00:00:00.000Z',
                 '2020-07-04T00:00:00.000Z',
                 '2020-07-05T00:00:00.000Z',
                 '2020-07-06T00:00:00.000Z',
                 '2020-07-07T00:00:00.000Z',
                 '2020-07-08T00:00:00.000Z',
                 '2020-07-09T00:00:00.000Z',
                 '2020-07-10T00:00:00.000Z',
                 '2020-07-11T00:00:00.000Z',
                 '2020-07-12T00:00:00.000Z',
                 '2020-07-13T00:00:00.000Z',
                 '2020-07-14T00:00:00.000Z',
                 '2020-07-15T00:00:00.000Z',
                 '2020-07-16T00:00:00.000Z',
                 '2020-07-17T00:00:00.000Z',
                 '2020-07-18T00:00:00.000Z',
                 '2020-07-19T00:00:00.000Z',
                 '2020-07-20T00:00:00.000Z',
                 '2020-07-21T00:00:00.000Z',
                 '2020-07-22T00:00:00.000Z',
                 '2020-07-23T00:00:00.000Z',
                 '2020-07-24T00:00:00.000Z',
                 '2020-07-25T00:00:00.000Z',
                 '2020-07-26T00:00:00.000Z',
                 '2020-07-27T00:00:00.000Z',
                 '2020-07-28T00:00:00.000Z',
                 '2020-07-29T00:00:00.000Z',
                 '2020-07-30T00:00:00.000Z',
                 '2020-07-31T00:00:00.000Z']

end_list =      ['2020-07-01T23:59:59.000Z',
                 '2020-07-02T23:59:59.000Z',
                 '2020-07-03T23:59:59.000Z',
                 '2020-07-04T23:59:59.000Z',
                 '2020-07-05T23:59:59.000Z',
                 '2020-07-06T23:59:59.000Z',
                 '2020-07-07T23:59:59.000Z',
                 '2020-07-08T23:59:59.000Z',
                 '2020-07-09T23:59:59.000Z',
                 '2020-07-10T23:59:59.000Z',
                 '2020-07-11T23:59:59.000Z',
                 '2020-07-12T23:59:59.000Z',
                 '2020-07-13T23:59:59.000Z',
                 '2020-07-14T23:59:59.000Z',
                 '2020-07-15T23:59:59.000Z',
                 '2020-07-16T23:59:59.000Z',
                 '2020-07-17T23:59:59.000Z',
                 '2020-07-18T23:59:59.000Z',
                 '2020-07-19T23:59:59.000Z',
                 '2020-07-20T23:59:59.000Z',
                 '2020-07-21T23:59:59.000Z',
                 '2020-07-22T23:59:59.000Z',
                 '2020-07-23T23:59:59.000Z',
                 '2020-07-24T23:59:59.000Z',
                 '2020-07-25T23:59:59.000Z',
                 '2020-07-26T23:59:59.000Z',
                 '2020-07-27T23:59:59.000Z',
                 '2020-07-28T23:59:59.000Z',
                 '2020-07-29T23:59:59.000Z',
                 '2020-07-30T23:59:59.000Z',
                 '2020-07-31T23:59:59.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("fromtweets-july2020.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-july2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-july2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7gg1yeuvp74q1exglrleurp4kd4t
Start Date:  2020-07-01T00:00:00.000Z
# of Tweets added from this response:  485
Total # of Tweets added:  485
-------------------
-------------------
Token:  b26v89c19zqg8o3fo7gg1yeuvp74q1exglrleurp4kd4t
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7gg1yeusm8trhitlhpxtu3a0iet9
Start Date:  2020-07-01T00:00:00.000Z
# of Tweets added from this response:  477
Total # of Tweets added:  962
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7gg26wkfuz7z5i9h389ix156w60t
Start Date:  2020-07-02T00:00:00.000Z
# of Tweets added from this response:  486
Total # of Tweets added:  1448
-------------------
-------------------
Token:  b26v89c19zqg8o3fo7gg26wkfuz7z5i9h389ix156w60t
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7gg26wkctqe9ymoxhkhvdg8xz7gd
Start Date:  2020-07-02T00:00:00.000Z
# of T

In [ ]:
#Inputs for tweets August 2020
headers = create_headers(bearer_token)
keyword = "-is:retweet -RT food emergency OR food crisis OR food aid OR food need OR food corona OR food rona OR food assistance OR food distribution OR food distribute OR food free OR WIC OR food security OR food insecurity OR food hungry OR #freefood OR #mealservice OR food pantry OR food bank OR covid food"
start_list =    ['2020-08-01T00:00:00.000Z',
                 '2020-08-02T00:00:00.000Z',
                 '2020-08-03T00:00:00.000Z',
                 '2020-08-04T00:00:00.000Z',
                 '2020-08-05T00:00:00.000Z',
                 '2020-08-06T00:00:00.000Z',
                 '2020-08-07T00:00:00.000Z',
                 '2020-08-08T00:00:00.000Z',
                 '2020-08-09T00:00:00.000Z',
                 '2020-08-10T00:00:00.000Z',
                 '2020-08-11T00:00:00.000Z',
                 '2020-08-12T00:00:00.000Z',
                 '2020-08-13T00:00:00.000Z',
                 '2020-08-14T00:00:00.000Z',
                 '2020-08-15T00:00:00.000Z',
                 '2020-08-16T00:00:00.000Z',
                 '2020-08-17T00:00:00.000Z',
                 '2020-08-18T00:00:00.000Z',
                 '2020-08-19T00:00:00.000Z',
                 '2020-08-20T00:00:00.000Z',
                 '2020-08-21T00:00:00.000Z',
                 '2020-08-22T00:00:00.000Z',
                 '2020-08-23T00:00:00.000Z',
                 '2020-08-24T00:00:00.000Z',
                 '2020-08-25T00:00:00.000Z',
                 '2020-08-26T00:00:00.000Z',
                 '2020-08-27T00:00:00.000Z',
                 '2020-08-28T00:00:00.000Z']

end_list =      ['2020-08-01T23:59:59.000Z',
                 '2020-08-02T23:59:59.000Z',
                 '2020-08-03T23:59:59.000Z',
                 '2020-08-04T23:59:59.000Z',
                 '2020-08-05T23:59:59.000Z',
                 '2020-08-06T23:59:59.000Z',
                 '2020-08-07T23:59:59.000Z',
                 '2020-08-08T23:59:59.000Z',
                 '2020-08-09T23:59:59.000Z',
                 '2020-08-10T23:59:59.000Z',
                 '2020-08-11T23:59:59.000Z',
                 '2020-08-12T23:59:59.000Z',
                 '2020-08-13T23:59:59.000Z',
                 '2020-08-14T23:59:59.000Z',
                 '2020-08-15T23:59:59.000Z',
                 '2020-08-16T23:59:59.000Z',
                 '2020-08-17T23:59:59.000Z',
                 '2020-08-18T23:59:59.000Z',
                 '2020-08-19T23:59:59.000Z',
                 '2020-08-20T23:59:59.000Z',
                 '2020-08-21T23:59:59.000Z',
                 '2020-08-22T23:59:59.000Z',
                 '2020-08-23T23:59:59.000Z',
                 '2020-08-24T23:59:59.000Z',
                 '2020-08-25T23:59:59.000Z',
                 '2020-08-26T23:59:59.000Z',
                 '2020-08-27T23:59:59.000Z',
                 '2020-08-28T23:59:59.000Z',
                 '2020-08-29T23:59:59.000Z',
                 '2020-08-30T23:59:59.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("fromtweets-aug2020.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-aug2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-aug2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7jg96edcrmqw3eslt7sv9kv2ewe5
Start Date:  2020-08-01T00:00:00.000Z
# of Tweets added from this response:  456
Total # of Tweets added:  456
-------------------
-------------------
Token:  b26v89c19zqg8o3fo7jg96edcrmqw3eslt7sv9kv2ewe5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7jg96e2uzw82zlyo2osjerdmrvr1
Start Date:  2020-08-01T00:00:00.000Z
# of Tweets added from this response:  460
Total # of Tweets added:  916
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7mc892supzlstl73yt4l4nnorzi5
Start Date:  2020-08-02T00:00:00.000Z
# of Tweets added from this response:  466
Total # of Tweets added:  1382
-------------------
-------------------
Token:  b26v89c19zqg8o3fo7mc892supzlstl73yt4l4nnorzi5
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fo7mc892i9wsrae97uqjzxucklcjul
Start Date:  2020-08-02T00:00:00.000Z
# of T

In [ ]:
#Inputs for tweets September 2020
headers = create_headers(bearer_token)
keyword = "-is:retweet -RT food emergency OR food crisis OR food aid OR food need OR food corona OR food rona OR food assistance OR food distribution OR food distribute OR food free OR WIC OR food security OR food insecurity OR food hungry OR #freefood OR #mealservice OR food pantry OR food bank OR covid food"
start_list =    ['2020-09-01T00:00:00.000Z',
                 '2020-09-02T00:00:00.000Z',
                 '2020-09-03T00:00:00.000Z',
                 '2020-09-04T00:00:00.000Z',
                 '2020-09-05T00:00:00.000Z',
                 '2020-09-06T00:00:00.000Z',
                 '2020-09-07T00:00:00.000Z',
                 '2020-09-08T00:00:00.000Z',
                 '2020-09-09T00:00:00.000Z',
                 '2020-09-10T00:00:00.000Z',
                 '2020-09-11T00:00:00.000Z',
                 '2020-09-12T00:00:00.000Z',
                 '2020-09-13T00:00:00.000Z',
                 '2020-09-14T00:00:00.000Z',
                 '2020-09-15T00:00:00.000Z',
                 '2020-09-16T00:00:00.000Z',
                 '2020-09-17T00:00:00.000Z',
                 '2020-09-18T00:00:00.000Z',
                 '2020-09-19T00:00:00.000Z',
                 '2020-09-20T00:00:00.000Z',
                 '2020-09-21T00:00:00.000Z',
                 '2020-09-22T00:00:00.000Z',
                 '2020-09-23T00:00:00.000Z',
                 '2020-09-24T00:00:00.000Z',
                 '2020-09-25T00:00:00.000Z',
                 '2020-09-26T00:00:00.000Z',
                 '2020-09-27T00:00:00.000Z',
                 '2020-09-28T00:00:00.000Z',
                 '2020-09-29T00:00:00.000Z',
                 '2020-09-30T00:00:00.000Z']

end_list =      ['2020-09-01T23:59:59.000Z',
                 '2020-09-02T23:59:59.000Z',
                 '2020-09-03T23:59:59.000Z',
                 '2020-09-04T23:59:59.000Z',
                 '2020-09-05T23:59:59.000Z',
                 '2020-09-06T23:59:59.000Z',
                 '2020-09-07T23:59:59.000Z',
                 '2020-09-08T23:59:59.000Z',
                 '2020-09-09T23:59:59.000Z',
                 '2020-09-10T23:59:59.000Z',
                 '2020-09-11T23:59:59.000Z',
                 '2020-09-12T23:59:59.000Z',
                 '2020-09-13T23:59:59.000Z',
                 '2020-09-14T23:59:59.000Z',
                 '2020-09-15T23:59:59.000Z',
                 '2020-09-16T23:59:59.000Z',
                 '2020-09-17T23:59:59.000Z',
                 '2020-09-18T23:59:59.000Z',
                 '2020-09-19T23:59:59.000Z',
                 '2020-09-20T23:59:59.000Z',
                 '2020-09-21T23:59:59.000Z',
                 '2020-09-22T23:59:59.000Z',
                 '2020-09-23T23:59:59.000Z',
                 '2020-09-24T23:59:59.000Z',
                 '2020-09-25T23:59:59.000Z',
                 '2020-09-26T23:59:59.000Z',
                 '2020-09-27T23:59:59.000Z',
                 '2020-09-28T23:59:59.000Z',
                 '2020-09-29T23:59:59.000Z',
                 '2020-09-30T23:59:59.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("fromtweets-sept2020.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-sept2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-sept2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fos5slkfw79lk20o7sq041l1qm7lod
Start Date:  2020-09-01T00:00:00.000Z
# of Tweets added from this response:  470
Total # of Tweets added:  470
-------------------
-------------------
Token:  b26v89c19zqg8o3fos5slkfw79lk20o7sq041l1qm7lod
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fos5slkfw47xfi1hkxocuh7f7o1b3x
Start Date:  2020-09-01T00:00:00.000Z
# of Tweets added from this response:  446
Total # of Tweets added:  916
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fos5t0d8a44b9t1sp79x3svtmf1l6l
Start Date:  2020-09-02T00:00:00.000Z
# of Tweets added from this response:  476
Total # of Tweets added:  1392
-------------------
-------------------
Token:  b26v89c19zqg8o3fos5t0d8a44b9t1sp79x3svtmf1l6l
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fos5t0d89y293upk2omevkfr8nsc59
Start Date:  2020-09-02T00:00:00.000Z
# of T

In [ ]:
#Inputs for tweets October 2020
headers = create_headers(bearer_token)
keyword = "-is:retweet -RT food emergency OR food crisis OR food aid OR food need OR food corona OR food rona OR food assistance OR food distribution OR food distribute OR food free OR WIC OR food security OR food insecurity OR food hungry OR #freefood OR #mealservice OR food pantry OR food bank OR covid food"
start_list =    ['2020-10-01T00:00:00.000Z',
                 '2020-10-02T00:00:00.000Z',
                 '2020-10-03T00:00:00.000Z',
                 '2020-10-04T00:00:00.000Z',
                 '2020-10-05T00:00:00.000Z',
                 '2020-10-06T00:00:00.000Z',
                 '2020-10-07T00:00:00.000Z',
                 '2020-10-08T00:00:00.000Z',
                 '2020-10-09T00:00:00.000Z',
                 '2020-10-10T00:00:00.000Z',
                 '2020-10-11T00:00:00.000Z',
                 '2020-10-12T00:00:00.000Z',
                 '2020-10-13T00:00:00.000Z',
                 '2020-10-14T00:00:00.000Z',
                 '2020-10-15T00:00:00.000Z',
                 '2020-10-16T00:00:00.000Z',
                 '2020-10-17T00:00:00.000Z',
                 '2020-10-18T00:00:00.000Z',
                 '2020-10-19T00:00:00.000Z',
                 '2020-10-20T00:00:00.000Z',
                 '2020-10-21T00:00:00.000Z',
                 '2020-10-22T00:00:00.000Z',
                 '2020-10-23T00:00:00.000Z',
                 '2020-10-24T00:00:00.000Z',
                 '2020-10-25T00:00:00.000Z',
                 '2020-10-26T00:00:00.000Z',
                 '2020-10-27T00:00:00.000Z',
                 '2020-10-28T00:00:00.000Z',
                 '2020-10-29T00:00:00.000Z',
                 '2020-10-30T00:00:00.000Z',
                 '2020-10-31T00:00:00.000Z']

end_list =      ['2020-10-01T23:59:59.000Z',
                 '2020-10-02T23:59:59.000Z',
                 '2020-10-03T23:59:59.000Z',
                 '2020-10-04T23:59:59.000Z',
                 '2020-10-05T23:59:59.000Z',
                 '2020-10-06T23:59:59.000Z',
                 '2020-10-07T23:59:59.000Z',
                 '2020-10-08T23:59:59.000Z',
                 '2020-10-09T23:59:59.000Z',
                 '2020-10-10T23:59:59.000Z',
                 '2020-10-11T23:59:59.000Z',
                 '2020-10-12T23:59:59.000Z',
                 '2020-10-13T23:59:59.000Z',
                 '2020-10-14T23:59:59.000Z',
                 '2020-10-15T23:59:59.000Z',
                 '2020-10-16T23:59:59.000Z',
                 '2020-10-17T23:59:59.000Z',
                 '2020-10-18T23:59:59.000Z',
                 '2020-10-19T23:59:59.000Z',
                 '2020-10-20T23:59:59.000Z',
                 '2020-10-21T23:59:59.000Z',
                 '2020-10-22T23:59:59.000Z',
                 '2020-10-23T23:59:59.000Z',
                 '2020-10-24T23:59:59.000Z',
                 '2020-10-25T23:59:59.000Z',
                 '2020-10-26T23:59:59.000Z',
                 '2020-10-27T23:59:59.000Z',
                 '2020-10-28T23:59:59.000Z',
                 '2020-10-29T23:59:59.000Z',
                 '2020-10-30T23:59:59.000Z',
                 '2020-10-31T23:59:59.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("fromtweets-oct2020.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-oct2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-oct2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fos8ssjxepfr0wem2kufa686anjlod
Start Date:  2020-10-01T00:00:00.000Z
# of Tweets added from this response:  479
Total # of Tweets added:  479
-------------------
-------------------
Token:  b26v89c19zqg8o3fos8ssjxepfr0wem2kufa686anjlod
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fos8ssjxekvgp2ezaofyi84wkndxml
Start Date:  2020-10-01T00:00:00.000Z
# of Tweets added from this response:  474
Total # of Tweets added:  953
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fos8t7ao2bq2egqtx40ql69mq1mw71
Start Date:  2020-10-02T00:00:00.000Z
# of Tweets added from this response:  469
Total # of Tweets added:  1422
-------------------
-------------------
Token:  b26v89c19zqg8o3fos8t7ao2bq2egqtx40ql69mq1mw71
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fos8t7ao244y7y44ibzlko9xdjhqil
Start Date:  2020-10-02T00:00:00.000Z
# of T

In [ ]:
#Inputs for tweets November 2020
headers = create_headers(bearer_token)
keyword = "-is:retweet -RT food emergency OR food crisis OR food aid OR food need OR food corona OR food rona OR food assistance OR food distribution OR food distribute OR food free OR WIC OR food security OR food insecurity OR food hungry OR #freefood OR #mealservice OR food pantry OR food bank OR covid food"
start_list =    ['2020-11-01T00:00:00.000Z',
                 '2020-11-02T00:00:00.000Z',
                 '2020-11-03T00:00:00.000Z',
                 '2020-11-04T00:00:00.000Z',
                 '2020-11-05T00:00:00.000Z',
                 '2020-11-06T00:00:00.000Z',
                 '2020-11-07T00:00:00.000Z',
                 '2020-11-08T00:00:00.000Z',
                 '2020-11-09T00:00:00.000Z',
                 '2020-11-10T00:00:00.000Z',
                 '2020-11-11T00:00:00.000Z',
                 '2020-11-12T00:00:00.000Z',
                 '2020-11-13T00:00:00.000Z',
                 '2020-11-14T00:00:00.000Z',
                 '2020-11-15T00:00:00.000Z',
                 '2020-11-16T00:00:00.000Z',
                 '2020-11-17T00:00:00.000Z',
                 '2020-11-18T00:00:00.000Z',
                 '2020-11-19T00:00:00.000Z',
                 '2020-11-20T00:00:00.000Z',
                 '2020-11-21T00:00:00.000Z',
                 '2020-11-22T00:00:00.000Z',
                 '2020-11-23T00:00:00.000Z',
                 '2020-11-24T00:00:00.000Z',
                 '2020-11-25T00:00:00.000Z',
                 '2020-11-26T00:00:00.000Z',
                 '2020-11-27T00:00:00.000Z',
                 '2020-11-28T00:00:00.000Z',
                 '2020-11-29T00:00:00.000Z',
                 '2020-11-30T00:00:00.000Z']

end_list =      ['2020-11-01T23:59:59.000Z',
                 '2020-11-02T23:59:59.000Z',
                 '2020-11-03T23:59:59.000Z',
                 '2020-11-04T23:59:59.000Z',
                 '2020-11-05T23:59:59.000Z',
                 '2020-11-06T23:59:59.000Z',
                 '2020-11-07T23:59:59.000Z',
                 '2020-11-08T23:59:59.000Z',
                 '2020-11-09T23:59:59.000Z',
                 '2020-11-10T23:59:59.000Z',
                 '2020-11-11T23:59:59.000Z',
                 '2020-11-12T23:59:59.000Z',
                 '2020-11-13T23:59:59.000Z',
                 '2020-11-14T23:59:59.000Z',
                 '2020-11-15T23:59:59.000Z',
                 '2020-11-16T23:59:59.000Z',
                 '2020-11-17T23:59:59.000Z',
                 '2020-11-18T23:59:59.000Z',
                 '2020-11-19T23:59:59.000Z',
                 '2020-11-20T23:59:59.000Z',
                 '2020-11-21T23:59:59.000Z',
                 '2020-11-22T23:59:59.000Z',
                 '2020-11-23T23:59:59.000Z',
                 '2020-11-24T23:59:59.000Z',
                 '2020-11-25T23:59:59.000Z',
                 '2020-11-26T23:59:59.000Z',
                 '2020-11-27T23:59:59.000Z',
                 '2020-11-28T23:59:59.000Z',
                 '2020-11-29T23:59:59.000Z',
                 '2020-11-30T23:59:59.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("fromtweets-nov2020.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-nov2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-nov2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosbtea5kvgc8b3ftfdvtlzhnqegl9
Start Date:  2020-11-01T00:00:00.000Z
# of Tweets added from this response:  454
Total # of Tweets added:  454
-------------------
-------------------
Token:  b26v89c19zqg8o3fosbtea5kvgc8b3ftfdvtlzhnqegl9
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosbtea5knv0gib7whqo79ttm88fi5
Start Date:  2020-11-01T00:00:00.000Z
# of Tweets added from this response:  447
Total # of Tweets added:  901
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosbteinae4ckodahmi5wihg0g9nr1
Start Date:  2020-11-02T00:00:00.000Z
# of Tweets added from this response:  452
Total # of Tweets added:  1353
-------------------
-------------------
Token:  b26v89c19zqg8o3fosbteinae4ckodahmi5wihg0g9nr1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosbteina80skubc3d1fjmgxe7rse5
Start Date:  2020-11-02T00:00:00.000Z
# of T

In [ ]:
#Inputs for tweets December 2020
headers = create_headers(bearer_token)
keyword = "-is:retweet -RT food emergency OR food crisis OR food aid OR food need OR food corona OR food rona OR food assistance OR food distribution OR food distribute OR food free OR WIC OR food security OR food insecurity OR food hungry OR #freefood OR #mealservice OR food pantry OR food bank OR covid food"
start_list =    ['2020-12-01T00:00:00.000Z',
                 '2020-12-02T00:00:00.000Z',
                 '2020-12-03T00:00:00.000Z',
                 '2020-12-04T00:00:00.000Z',
                 '2020-12-05T00:00:00.000Z',
                 '2020-12-06T00:00:00.000Z',
                 '2020-12-07T00:00:00.000Z',
                 '2020-12-08T00:00:00.000Z',
                 '2020-12-09T00:00:00.000Z',
                 '2020-12-10T00:00:00.000Z',
                 '2020-12-11T00:00:00.000Z',
                 '2020-12-12T00:00:00.000Z',
                 '2020-12-13T00:00:00.000Z',
                 '2020-12-14T00:00:00.000Z',
                 '2020-12-15T00:00:00.000Z',
                 '2020-12-16T00:00:00.000Z',
                 '2020-12-17T00:00:00.000Z',
                 '2020-12-18T00:00:00.000Z',
                 '2020-12-19T00:00:00.000Z',
                 '2020-12-20T00:00:00.000Z',
                 '2020-12-21T00:00:00.000Z',
                 '2020-12-22T00:00:00.000Z',
                 '2020-12-23T00:00:00.000Z',
                 '2020-12-24T00:00:00.000Z',
                 '2020-12-25T00:00:00.000Z',
                 '2020-12-26T00:00:00.000Z',
                 '2020-12-27T00:00:00.000Z',
                 '2020-12-28T00:00:00.000Z',
                 '2020-12-29T00:00:00.000Z',
                 '2020-12-30T00:00:00.000Z',
                 '2020-12-31T00:00:00.000Z']

end_list =      ['2020-12-01T23:59:59.000Z',
                 '2020-12-02T23:59:59.000Z',
                 '2020-12-03T23:59:59.000Z',
                 '2020-12-04T23:59:59.000Z',
                 '2020-12-05T23:59:59.000Z',
                 '2020-12-06T23:59:59.000Z',
                 '2020-12-07T23:59:59.000Z',
                 '2020-12-08T23:59:59.000Z',
                 '2020-12-09T23:59:59.000Z',
                 '2020-12-10T23:59:59.000Z',
                 '2020-12-11T23:59:59.000Z',
                 '2020-12-12T23:59:59.000Z',
                 '2020-12-13T23:59:59.000Z',
                 '2020-12-14T23:59:59.000Z',
                 '2020-12-15T23:59:59.000Z',
                 '2020-12-16T23:59:59.000Z',
                 '2020-12-17T23:59:59.000Z',
                 '2020-12-18T23:59:59.000Z',
                 '2020-12-19T23:59:59.000Z',
                 '2020-12-20T23:59:59.000Z',
                 '2020-12-21T23:59:59.000Z',
                 '2020-12-22T23:59:59.000Z',
                 '2020-12-23T23:59:59.000Z',
                 '2020-12-24T23:59:59.000Z',
                 '2020-12-25T23:59:59.000Z',
                 '2020-12-26T23:59:59.000Z',
                 '2020-12-27T23:59:59.000Z',
                 '2020-12-28T23:59:59.000Z',
                 '2020-12-29T23:59:59.000Z',
                 '2020-12-30T23:59:59.000Z',
                 '2020-12-31T23:59:59.000Z']
max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("fromtweets-dec2020.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 500 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-dec2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "fromtweets-dec2020.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foset6pcpe5upyvqo2b00pcsit56d9
Start Date:  2020-12-01T00:00:00.000Z
# of Tweets added from this response:  476
Total # of Tweets added:  476
-------------------
-------------------
Token:  b26v89c19zqg8o3foset6pcpe5upyvqo2b00pcsit56d9
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3foset6pceuv2lsmde40alvhadqvsl9
Start Date:  2020-12-01T00:00:00.000Z
# of Tweets added from this response:  482
Total # of Tweets added:  958
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosetlg3d51b7na75w3ic7jrqf9dod
Start Date:  2020-12-02T00:00:00.000Z
# of Tweets added from this response:  474
Total # of Tweets added:  1432
-------------------
-------------------
Token:  b26v89c19zqg8o3fosetlg3d51b7na75w3ic7jrqf9dod
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fosetlg3d3gzclj4ntz1ki4dmfdu9p
Start Date:  2020-12-02T00:00:00.000Z
# of T